# Exercice 6.

This netbook contains the results for the first and second problem of this exercice.

### Problem 1.

In [1]:
def crc(msg, div, code='00000000'):
    """Cyclic Redundancy Check
    Generates an error detecting code based on an inputted message
    and divisor in the form of a polynomial representation.
    Arguments:
        msg: The input message of which to generate the output code.
        div: The divisor in polynomial form. For example, if the polynomial
            of x^3 + x + 1 is given, this should be represented as '1011' in
            the div argument.
        code: This is an option argument where a previously generated code may
            be passed in. This can be used to check validity. If the inputted
            code produces an outputted code of all zeros, then the message has
            no errors.
    Returns:
        An error-detecting code generated by the message and the given divisor.
    """
    # Append the code to the message. If no code is given, default to '00000000'
    msg = msg + code

    # Convert msg and div into list form for easier handling
    msg = list(msg)
    div = list(div)

    # Loop over every message bit (minus the appended code)
    for i in range(len(msg)-len(code)):
        # If that messsage bit is 1, perform modulo 2 multiplication
        if msg[i] == '1':
            for j in range(len(div)):
                # Perform modulo 2 multiplication on each index of the divisor
                msg[i+j] = str((int(msg[i+j])+int(div[j]))%2)
    
    # Output the last error-checking code portion of the message generated
    return ''.join(msg[-len(code):])

In [2]:
def wep_cipher(text, key, init_vector):
    """
    Arguments:
        text -- information to be ciphered or deciphered
        key  -- Key used
        init_vector  -- IV

    Returns:
        result -- ciphered or deciphered text
    """
    PRNG = pseudo_random_noise_generators(key, init_vector, text)
    result = xor(PRNG, text)

    return result

In [3]:
def pseudo_random_noise_generators(key, init_vector, text):
    """
    Arguments:
        key  -- Key used
        init_vector  -- IV
        text -- information to be ciphered

    Returns: String of the output bits
    """

    PO = []
    PS = []
    
    for i in range(0, len(text)):
        if i == 0:
            # PS(0) = init_vector (+) key
            PS = list(xor(init_vector, key))
        
        else:
            prev_PS = PS.copy()
            for j in range(0, len(PS)):
                
                if j == 0:
                    # PS(i)[0] = PS(i − 1)[4] (+) PS(i − 1)[7]
                    PS[j] = xor(prev_PS[4], prev_PS[7])
                else:
                    # PS(i)[j] = PS(i−1)[j−1], j>0
                    PS[j] = prev_PS[j-1]
                
        # PO[i] = PS(i)[4] (+) PS(i)[7]
        PO.append(xor(PS[4], PS[7]))
        
    return "".join(PO)


In [4]:
def xor(num1, num2):
    """XOR two binary numbers."""
    # Same bit = 0 diferent bit = 1
    y = int(num1, 2)^int(num2,2)

    max_size = max(len(num1),len(num2))
    xor_result = bin(y)[2:].zfill(max_size)

    return xor_result

In [5]:
def hex_to_bin(hexdec):
    """Convert hexadecimal to binary."""
    return bin(int(hexdec, 16))[2:]        

### 1.1

Probe that the ciphered data for the clear text **data 0x0123**, key=0x33 and IV=0x11 **is: 0x667E92**

In [6]:
text_data = "0x0123"
key = "0x33"
iv = "0x11"
exp_chiper = "0x667E92"

First we convert all the hexadecimal variables into binary.

In [7]:
binary_text_data = hex_to_bin(text_data).zfill(16)
binary_text_data

'0000000100100011'

In [8]:
binary_key = hex_to_bin(key).zfill(8)
binary_key

'00110011'

In [9]:
binary_iv = hex_to_bin(iv).zfill(8)    
binary_iv

'00010001'

In [10]:
binary_exp_cipher = hex_to_bin(exp_chiper).zfill(24)
binary_exp_cipher

'011001100111111010010010'

#### CRC Length = x^8 + 1

We know that the CRC length is:

In [11]:
crc_length = "100000001"

#### CRC Calculation

Then we compute the CRC code for the binary text data:

In [12]:
crc_code = crc(binary_text_data, crc_length)
crc_code

'00100010'

#### Chipher

Next, the text data is chipered using the previous obtained CRC code and the WEP chiper:

In [14]:
text_to_cipher = binary_text_data + crc_code

In [15]:
ciphered_text = wep_cipher(text_to_cipher, binary_key, binary_iv)

#### Validation

To probe the obtained ciphered text is compared with the binary expected value:

In [17]:
ciphered_text == binary_exp_cipher

True

The values are equal, so we proved that the text data **0x0123** ciphered is **0x667E92**.

### 1.2

Probe how deciphering with a wrong key (key=0x22) an error condition is reported

#### Validation with correct key

In [20]:
dec_text = wep_cipher(ciphered_text, binary_key, binary_iv)
print('CRC Correct:', crc(dec_text[:16], crc_length, crc_code) == '00000000')

CRC Correct: True


#### Validation with wrong key

In [21]:
dec_text = wep_cipher(ciphered_text, hex_to_bin("0x22").zfill(8), binary_iv)
print('CRC Correct:', crc(dec_text[:16], crc_length, crc_code) == '00000000')

CRC Correct: False


As we can see with the wrong key the obtained result from the crc method is different.

### Problem 2.